#folium

In [ ]:
import folium

In [ ]:
import pandas as pd
df_1 = pd.read_excel('/content/sample_data/jejuMatJip.xlsx')
df_2 = pd.read_excel('/content/sample_data/jejudoMatJip.xlsx')
df_3 = pd.read_excel('/content/sample_data/jejuYeoHang.xlsx')
df_4 = pd.read_excel('/content/sample_data/jejudoGwanGwang.xlsx')

In [ ]:
df_place=list(df_1.place.unique())+list(df_2.place.unique())+list(df_3.place.unique())+list(df_4.place.unique())

In [ ]:
len(df_place)

1427

In [ ]:
# 카카오API를 사용하여 주소->좌표 변환
import requests, json

def get_location(address):
  url = 'https://dapi.kakao.com/v2/local/search/address.json?query=' + address
  # 'KaKaoAK '는 그대로 두시고 개인키만 지우고 입력해 주세요.
  # ex) KakaoAK 6af8d4826f0e56c54bc794fa8a294
#  headers = {"Authorization": "KakaoAK 개인키"}
  headers = {"Authorization": "KakaoAK aec61e1d1ba5fe57f9f537d020297cf0"}
  api_json = json.loads(str(requests.get(url,headers=headers).text))
  address = api_json['documents'][0]['address']
  crd = {"lat": str(address['y']), "lng": str(address['x'])}
  address_name = address['address_name']

  return crd

In [ ]:
from geopy.geocoders import Nominatim

def geocoding(address):
    geolocoder = Nominatim(user_agent = 'South Korea', timeout=None)
    geo = geolocoder.geocode(address)
    crd = {"lat": str(geo.latitude), "lng": str(geo.longitude)}

    return crd



In [ ]:
import numpy as np
crd = []
for i in range(len(df_place)):
  try :
      crd.append(geocoding(df_place[i]))
  except:
    crd.append(np.nan) 

In [ ]:
len(crd)

1427

In [ ]:
crd

[{'lat': '33.2572971', 'lng': '126.515299'},
 {'lat': '37.220334949999994', 'lng': '127.31538605869258'},
 nan,
 {'lat': '35.1799528', 'lng': '129.0752365'},
 {'lat': '46.3144754', 'lng': '11.0480288'},
 nan,
 {'lat': '33.3939924', 'lng': '126.5626653'},
 nan,
 {'lat': '33.3939924', 'lng': '126.5626653'},
 nan,
 {'lat': '33.3663086', 'lng': '126.3574576'},
 {'lat': '33.2783709', 'lng': '126.71240130847076'},
 nan,
 nan,
 {'lat': '33.38034745', 'lng': '126.54825445912047'},
 nan,
 nan,
 {'lat': '33.2505828', 'lng': '126.4232425'},
 nan,
 {'lat': '37.476814700000006', 'lng': '126.72353818699722'},
 nan,
 nan,
 nan,
 nan,
 {'lat': '33.2644805', 'lng': '126.3704345'},
 {'lat': '33.45079', 'lng': '126.3752'},
 {'lat': '35.87085015', 'lng': '128.5982098265411'},
 nan,
 nan,
 nan,
 nan,
 {'lat': '33.1997201', 'lng': '126.29021937499999'},
 nan,
 {'lat': '34.7810635', 'lng': '128.5638955'},
 {'lat': '-6.0287767', 'lng': '-79.6249366'},
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 {'lat': '33.458

In [ ]:
df = pd.DataFrame({'지역명' : df_place, '위치' : crd})

In [ ]:
df2 = df[df.위치.isna() != 1]

In [ ]:
df2 = df2.reset_index()
df2

,index,지역명,위치
0,0,Seogwipo,"{'lat': '33.2572971', 'lng': '126.515299'}"
1,1,청초수물회&섭국 [신관],"{'lat': '37.220334949999994', 'lng': '127.3153..."
2,3,"Busan, South Korea","{'lat': '35.1799528', 'lng': '129.0752365'}"
3,4,NaN,"{'lat': '46.3144754', 'lng': '11.0480288'}"
4,6,Jeju,"{'lat': '33.3939924', 'lng': '126.5626653'}"
...,...,...,...
507,1417,한라생태숲,"{'lat': '33.43158435', 'lng': '126.60092741157..."
508,1418,용연구름다리,"{'lat': '33.5149038', 'lng': '126.5141851'}"
509,1420,관덕정,"{'lat': '37.583299600000004', 'lng': '126.9946..."
510,1422,엉또폭포,"{'lat': '33.268431050000004', 'lng': '126.5000..."


In [ ]:
df2 = df2.drop('index', axis = 1)

In [ ]:
df2.to_csv('jejumap.csv')

In [ ]:
df2['위도'] = 0
df2['경도'] = 0
for i in range(len(df2.위치)):
    df2['위도'].iloc[i] = float(df2.위치[:][i]['lat'])
    df2['경도'].iloc[i] = float(df2.위치[:][i]['lng'])


/usr/local/lib/python3.7/dist-packages/pandas/core/indexing.py:1732: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_single_block(indexer, value, name)


In [ ]:
jejumap = folium.Map(location = [33.26,126.5],zoom_start=12)
for name, lat, lng in zip(df2.지역명, df2.위도, df2.경도):
    folium.Marker([lat, lng], popup=name).add_to(jejumap)

# 지도를 HTML 파일로 저장하기
jejumap.save('./top5.html')

In [ ]:
df3 = df2[((df2.위도 >= 33.11) & (df2.위도 <= 34.0)) & ((df2.경도 >= 126.0) & (df2.경도 <= 127.0))]

In [ ]:
# 북위 331127～333350, 동경 1260843″～1265820

In [ ]:
df3 = df3.reset_index().drop('index', axis = 1)

In [ ]:
df3

,지역명,위치,위도,경도
0,Seogwipo,"{'lat': '33.2572971', 'lng': '126.515299'}",33.257297,126.515299
1,Jeju,"{'lat': '33.3939924', 'lng': '126.5626653'}",33.393992,126.562665
2,Jeju-do,"{'lat': '33.3939924', 'lng': '126.5626653'}",33.393992,126.562665
3,새별오름,"{'lat': '33.3663086', 'lng': '126.3574576'}",33.366309,126.357458
4,"Namwon Seogwipo Jeju,South korea","{'lat': '33.2783709', 'lng': '126.712401308470...",33.278371,126.712401
...,...,...,...,...
370,서귀포 이마트,"{'lat': '33.248332899999994', 'lng': '126.5090...",33.248333,126.509085
371,한라생태숲,"{'lat': '33.43158435', 'lng': '126.60092741157...",33.431584,126.600927
372,용연구름다리,"{'lat': '33.5149038', 'lng': '126.5141851'}",33.514904,126.514185
373,엉또폭포,"{'lat': '33.268431050000004', 'lng': '126.5000...",33.268431,126.500086


In [ ]:
df3['color'] = ''
df3['dataframe'] = 0
for i in range(len(df3)):
  if df3.지역명.iloc[i] in list(df_1.place.unique()) :
    df3['dataframe'].iloc[i] = 1
    df3['color'].iloc[i] = 'blue'
  elif df3.지역명.iloc[i] in list(df_2.place.unique()) :
    df3['dataframe'].iloc[i]  = 2
    df3['color'].iloc[i] = 'orange'
  elif df3.지역명.iloc[i] in list(df_3.place.unique()):
    df3['dataframe'].iloc[i]  = 3
    df3['color'].iloc[i] = 'red'
  elif df3.지역명.iloc[i] in list(df_4.place.unique()):
    df3['dataframe'].iloc[i]  = 4
    df3['color'].iloc[i] = 'green'
  else :
    df3['color'].iloc[i] = 'black'

/usr/local/lib/python3.7/dist-packages/pandas/core/indexing.py:1732: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_single_block(indexer, value, name)


In [ ]:
df_list = []
for i in range(len(df3)):
  if df3.dataframe.iloc[i] == 1:
    df_list.append(df_1[df_1.place == df3.지역명.iloc[i]])
  elif df3.dataframe.iloc[i] == 2:
    df_list.append(df_2[df_2.place == df3.지역명.iloc[i]])
  elif df3.dataframe.iloc[i] == 3:
    df_list.append(df_3[df_3.place == df3.지역명.iloc[i]])
  elif df3.dataframe.iloc[i] == 4:
    df_list.append(df_4[df_4.place == df3.지역명.iloc[i]])


In [ ]:
df3

,지역명,위치,위도,경도,color,dataframe,like
0,Seogwipo,"{'lat': '33.2572971', 'lng': '126.515299'}",33.257297,126.515299,blue,1,0
1,Jeju,"{'lat': '33.3939924', 'lng': '126.5626653'}",33.393992,126.562665,blue,1,0
2,Jeju-do,"{'lat': '33.3939924', 'lng': '126.5626653'}",33.393992,126.562665,blue,1,0
3,새별오름,"{'lat': '33.3663086', 'lng': '126.3574576'}",33.366309,126.357458,blue,1,0
4,"Namwon Seogwipo Jeju,South korea","{'lat': '33.2783709', 'lng': '126.712401308470...",33.278371,126.712401,blue,1,0
...,...,...,...,...,...,...,...
370,서귀포 이마트,"{'lat': '33.248332899999994', 'lng': '126.5090...",33.248333,126.509085,green,4,0
371,한라생태숲,"{'lat': '33.43158435', 'lng': '126.60092741157...",33.431584,126.600927,green,4,0
372,용연구름다리,"{'lat': '33.5149038', 'lng': '126.5141851'}",33.514904,126.514185,red,3,0
373,엉또폭포,"{'lat': '33.268431050000004', 'lng': '126.5000...",33.268431,126.500086,green,4,0


In [ ]:

df3['like'] = 0
for i in range(len(df3)):
  if type(df_list[i].like.values[0]) == str:
    temp=df_list[i].like.str.replace(',','')
    temp[temp.isna()!=0] = '0'
    df3['like'].iloc[i] = temp.astype('int').sum()


In [ ]:
temp = df3.drop_duplicates(['지역명']).reset_index().drop('index',axis = 1)
temp

,지역명,위치,위도,경도,color,dataframe,like
0,Seogwipo,"{'lat': '33.2572971', 'lng': '126.515299'}",33.257297,126.515299,blue,1,1294
1,Jeju,"{'lat': '33.3939924', 'lng': '126.5626653'}",33.393992,126.562665,blue,1,14388
2,Jeju-do,"{'lat': '33.3939924', 'lng': '126.5626653'}",33.393992,126.562665,blue,1,1480
3,새별오름,"{'lat': '33.3663086', 'lng': '126.3574576'}",33.366309,126.357458,blue,1,7
4,"Namwon Seogwipo Jeju,South korea","{'lat': '33.2783709', 'lng': '126.712401308470...",33.278371,126.712401,blue,1,35
...,...,...,...,...,...,...,...
236,연북정,"{'lat': '33.5400778', 'lng': '126.636073489833...",33.540078,126.636073,green,4,45
237,서귀포 이마트,"{'lat': '33.248332899999994', 'lng': '126.5090...",33.248333,126.509085,green,4,140
238,한라생태숲,"{'lat': '33.43158435', 'lng': '126.60092741157...",33.431584,126.600927,green,4,59
239,엉또폭포,"{'lat': '33.268431050000004', 'lng': '126.5000...",33.268431,126.500086,green,4,56


In [ ]:
df4 = temp.drop_duplicates(['위도','경도']).reset_index().drop('index',axis = 1)

In [ ]:
from folium.plugins import MarkerCluster
jejumap = folium.Map(location = [33.26,126.5],zoom_start=12)
marker_cluster = MarkerCluster().add_to(jejumap)
for name, lat, lng, color,like in zip(df4.지역명, df4.위도, df4.경도, df4.color,df4.like):
    folium.Marker([lat, lng], popup=folium.Popup(f'지역명 : {name}<br> 좋아요 수 :{like}', min_width=200, max_width=200),icon=folium.Icon(color = color)).add_to(marker_cluster)

# 지도를 HTML 파일로 저장하기
jejumap.save('./jejumap.html')


In [ ]:
df3.to_csv('jeju.csv')